## 载入python库

In [72]:
# 载入python库
import pandas as pd
import xlrd, openpyxl
import re
import os
from copy import deepcopy
import csv
import pickle as pkl
import json
from tqdm.notebook import tqdm, tnrange
from multiprocessing.dummy import Pool 

##  排除数据源中的干扰行函数

In [6]:
#排除数据源中的干扰行
def fliter_none(issue):
    pattern = re.compile('\d{6}.[A-Z]{2}')
    try:
        id_stock = pattern.search(issue)
        if id_stock is not None :
            return True
        else:
            return False
    except:
        return False 

## 创建文件夹函数

In [7]:
#创建所需的文件夹
def mkdir(xlsx_filepath):
    root_path = '/'.join(xlsx_filepath.split('/')[:-1])
    
    csv_filepath = root_path + '/StockData_csv'
    fund_filepath = root_path + '/StockData_fund'
    point_filepath =  root_path + '/StockData_point'
    weighted_stock_filepath = root_path + '/StockData_weighted_stock'
    important_stock_filepath = root_path + '/StockData_important_stock'
    total_filepath = root_path + '/StockData_total_stock'
    
    filepath_list = [xlsx_filepath,csv_filepath,fund_filepath,point_filepath,
                     weighted_stock_filepath,important_stock_filepath,total_filepath]
    
    for filepath in filepath_list:
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        print('已建立', filepath)
    
    return filepath_list

## 分别生成每只基金重仓股即其持股比例的列表

In [8]:
#分别生成每只基金重仓股即其持股比例的列表
def generate_dict(dataframe, node_network_csv_path, fund_network_pkl_path):
    
    fund_network = {}
    node_network = {'node': [], 'group': [], 'k_value': []}  #生成一个储存node信息的字典
    for _, row in dataframe.iterrows():
        fund, stock, group, fluent_rate, value_rate, k_value = row
        if fund in fund_network:
            fund_network[fund].append((stock, k_value))
        else:
            fund_network[fund] = []
            fund_network[fund].append((stock, k_value))
            node_network['node'].append(fund)
            node_network['group'].append(group)
    for fund in node_network['node']:
        node_network['k_value'].append(fund_network[fund])
    node_network = pd.DataFrame(node_network)
    node_network.to_csv(node_network_csv_path, encoding='utf_8_sig',index=True)
    
    with open(fund_network_pkl_path, 'wb') as f:   #with：上下文管理器
        pkl.dump(fund_network, f)
    
    return fund_network  #{基金：[(股票1:k1),(股票2：k2)，……]}基金持仓各个股票及k值列表，列表内元素为元组

## 统计两只基金相同持股的股票列表，若比例大于临界值则记为1，否则记为0

In [9]:
#统计两只基金相同持股的股票列表，若比例大于临界值则记为1，否则记为0
def find_common(stock_list1, stock_list2, threshold):
    coshare_element = []
    for stock_name1, rate1 in stock_list1:  #[(股票1:k1)，(股票2：k2)，……]列表内每个元素由两部分组成
        for stock_name2, rate2 in stock_list2:
            if (stock_name1 == stock_name2) and ((rate1 > threshold) & (rate2>threshold)):
                coshare_element.append((stock_name1, 1))
            elif (stock_name1 == stock_name2) and ((rate1 <= threshold) | (rate2<=threshold)):
                coshare_element.append((stock_name1, 0))
    return coshare_element   #[(股票1:1)，(股票2:1)，(股票3:0)，……]

## 统计两只基金相同持股的股票数量及均大于临界值的股票数量

In [10]:
#统计两只基金相同持股的股票数量及均大于临界值的股票数量
def generate_network_dataframe(fund_network, fund_network_copy, threshold):
    network_dataframe = {'source':[],'target':[],'coshare_stock':[],'coshare_num':[],'weight_stock':[],'weight':[]}
    
    for key1, stock_list1 in fund_network.items():
        del fund_network_copy[key1]  #把自己删掉避免自己和自己比较
        for key2, stock_list2 in fund_network_copy.items():
            coshare_element = find_common(stock_list1, stock_list2, threshold) #[(股票1:1)，(股票2:1)，(股票3:0)，……]
            
            
            if len(coshare_element)>0:
                network_dataframe['source'].append(key1)
                network_dataframe['target'].append(key2)
                network_dataframe['coshare_stock'].append('*'.join(sorted([share_stock[0] for share_stock in coshare_element])))
                network_dataframe['coshare_num'].append(len(coshare_element))
                network_dataframe['weight_stock'].append('*'.join(sorted([share_stock[0] for share_stock in coshare_element if share_stock[1]==1])))
                network_dataframe['weight'].append(sum([share_stock[1] for share_stock in coshare_element]))
                
                
    network_dataframe = pd.DataFrame(network_dataframe)                            #return 这个是全weight数据
    network_dataframe_weight = network_dataframe[network_dataframe['weight']!=0]   #return 这个是删除weight为0的
    
    output_list = ['source','target','weight_stock','weight']
    return network_dataframe_weight[output_list]

## 共同持有重仓股排名列表

In [11]:
#共同持有重仓股排名列表
def generate_weighted_stock(fund_network, fund_network_copy, threshold):
    weighted_stock = {}  ##形成重仓股排名列表  {股票1：3，股票2：6,……}
    for key1, stock_list1 in fund_network.items():
        del fund_network_copy[key1]  #把自己删掉避免自己和自己比较
        for key2, stock_list2 in fund_network_copy.items():
            coshare_element = find_common(stock_list1, stock_list2, threshold) #[(股票1:1)，(股票2:1)，(股票3:0)，……]
            
            for share_stock in coshare_element:
                if share_stock[1]==1:
                    if share_stock[0] in weighted_stock.keys():
                        weighted_stock[share_stock[0]] +=1
                    else:
                        weighted_stock[share_stock[0]] = 1
    weighted_stock_order = sorted(weighted_stock.items(),key=lambda x:x[1],reverse=False)                  
    
    return weighted_stock_order           

## 单独持有重仓股排名列表

In [12]:
#单独持有重仓股排名列表
def generate_important_stock(fund_network,threshold):
    important_stock = {} #形成重仓股排名列表  {股票1：3，股票2：6,……} 
    
    for key1, stock_list1 in fund_network.items():  #{基金：[(股票1:k1),(股票2：k2)，……]}
        for stock_name1, rate1 in stock_list1:      #[(股票1:k1)，(股票2：k2)，……]列表内每个元素由两部分组成
            if rate1 > threshold:
                if stock_name1 in important_stock.keys():
                    important_stock[stock_name1] += 1
                else:
                    important_stock[stock_name1] = 1
      
    important_stock_order = sorted(important_stock.items(),key=lambda x:x[1],reverse=False)                  
    
    return important_stock_order
        

## 生成并行的分块数据

In [13]:
def sub_list(init_list, number_thread):
    end_list = [list(i) for i in zip(*zip(*[iter(init_list)] *number_thread))]
    count = len(init_list)%number_thread
    if count!=0:   
        for k,v in enumerate(init_list[-count:]):
            end_list[k].append(v) 
    return end_list

## 生成网络结构数据

In [68]:
#生成网络结构数据
def xlsx_csv_fund(xlsx_filepath, hyper_params, hyper_threshold, invest_category = None):
    
    #创建所需文件夹
    xlsx_filepath,csv_filepath,fund_filepath,point_filepath, \
    weighted_stock_filepath,important_stock_filepath,total_filepath = mkdir(xlsx_filepath)
    
    #获取文件路径
    list_dir =  [os.path.splitext(dir_) for dir_ in os.listdir(xlsx_filepath)]
    
    #确认成的网络并且生成label
    if invest_category is not None:
        print('正在生成'+'+'.join(invest_category)+'投资的网络')
    else:
        print('正在生成全部投资的网络')
        
    total_weighted_stock = {}
    total_important_stock ={}
    for tuple_dir in tqdm(list_dir):   #逐次针对当前日期
        #读取wind数据
        date_name, format_name = tuple_dir
        dataframe = pd.read_excel(xlsx_filepath+'/'+date_name+format_name)
        
        #删去不符合投资类型的股票，并检验是否数据框为空，为空则跳转到下一个循环
        if invest_category is not None:
            dataframe = dataframe[dataframe['投资类型'].isin(invest_category)]
        if dataframe.empty:
            continue
        
        #如果存在csv文件则直接读取，如果不存在则生成csv
        csv_name = ('_').join(date_name.split('-'))
        data_csv_path = csv_filepath+'/'+csv_name+'.csv'
        if not os.path.exists(data_csv_path):
            #从原始数据提取必要信息
            column_name = ['代码','名称','股票代码','股票简称','持股占流通股比(%)','持股市值占基金净值比(%)','管理公司']
            data_csv = dataframe[dataframe['代码'].apply(fliter_none)][column_name]
            data_csv = data_csv.rename(columns = {'代码':'code',
                                                  '名称':'source',
                                                  '股票代码':'stock_code',
                                                  '股票简称':'target', 
                                                  '持股占流通股比(%)':'fluent_rate',
                                                  '持股市值占基金净值比(%)':'value_rate', 
                                                  '管理公司':'group'})
            dataframe_fund = data_csv[['source','target','group','fluent_rate','value_rate']]
            
            #生成k值进一步完善dataframe_fund
            a, b, c = hyper_params
            dataframe_fund_copy = dataframe_fund.copy()
            #生成k值
            dataframe_fund_copy['k_value'] = dataframe_fund.apply(lambda row: round(a*row[3]+b*row[4]+c*row[3]*row[4], 2), axis=1)
            dataframe_fund_copy.to_csv(data_csv_path, encoding='utf_8_sig', index=True)
        else:
            print('已存在',csv_name)
            dataframe_fund_copy = pd.read_csv(data_csv_path, index_col = 0)
        
        #生成基金字典并储存基金网络
        pkl_filepath = point_filepath + '/Pickle'
        node_network_csv_path = point_filepath + '/' + csv_name + '_node_network' +'.csv'
        fund_network_pkl_path = pkl_filepath +  '/' + csv_name + '_fund_network' +'.pkl'
        if not os.path.exists(pkl_filepath):
            os.makedirs(pkl_filepath)
        if os.path.exists(node_network_csv_path):
            with open(fund_network_pkl_path,'rb') as f:   
                fund_network = pkl.load(f)   
        else:   
            fund_network = generate_dict(dataframe_fund_copy, node_network_csv_path, fund_network_pkl_path)
        fund_network_copy = deepcopy(fund_network)
        threshold = hyper_threshold
        
        #生成制作网络的数据表
        network_path = fund_filepath+'/'+csv_name+'.csv'
        if not os.path.exists(network_path):
            network_dataframe = generate_network_dataframe(fund_network,fund_network_copy,threshold)
            network_dataframe.to_csv(network_path, encoding='utf_8_sig', index=True)
        else:
            network_dataframe = pd.read_csv(network_path, index_col = 0)

        #生成共同重仓股列表
        json_weighted_filepath = weighted_stock_filepath + '/Json'
        weighted_order_csv_path = weighted_stock_filepath+'/'+csv_name+'.csv'
        weighted_order_json_path = json_weighted_filepath+'/'+csv_name+'.json'   
        fund_network_copy = deepcopy(fund_network)
        if not os.path.exists(json_weighted_filepath):
            os.makedirs(json_weighted_filepath)
        if not os.path.exists(weighted_order_csv_path):
            weighted_stock_dataframe = generate_weighted_stock(fund_network,fund_network_copy,threshold)
            with open(weighted_order_csv_path, 'w', encoding = 'utf_8_sig',newline='') as f:     
                csv_write = csv.writer(f)
                for new_line in weighted_stock_dataframe:
                    csv_write.writerow(new_line) 
            with open(weighted_order_json_path,'w', encoding = 'utf_8_sig') as f:   
                f.write(json.dumps(weighted_stock_dataframe))
        else:
            with open(weighted_order_json_path,'r', encoding = 'utf_8_sig') as f:   
                weighted_stock_dataframe = f.read()
                               ####用此方法可以将字典里的元素逐行读入；否则生成一个横向很长的列表。（转置作用）

        #生成单独重仓股列表
        json_important_filepath = important_stock_filepath + '/Json'
        important_stock_csv_path = important_stock_filepath+'/'+csv_name+'.csv'
        important_order_json_path = json_important_filepath+'/'+csv_name+'.json' 
        if not os.path.exists(json_important_filepath):
            os.makedirs(json_important_filepath)
        if not os.path.exists(important_stock_csv_path):
            important_stock_dataframe = generate_important_stock(fund_network,threshold)
            with open(important_stock_csv_path, 'w', encoding = 'utf_8_sig',newline='') as f:     
                csv_write = csv.writer(f)
                for new_line in important_stock_dataframe:
                    csv_write.writerow(new_line)   
            with open(important_order_json_path,'w', encoding = 'utf_8_sig') as f:   
                f.write(json.dumps(important_stock_dataframe))  
        else:
            with open(important_order_json_path,'r', encoding = 'utf_8_sig') as f:   
                important_stock_dataframe = f.read()

        #将共同重仓股信息汇总在一张表
        total_weighted_stock[csv_name]=[]  ###注意必须先定义字典的值为一个列表，才能向列表里加元素!!!
        for tuple_stock in weighted_stock_dataframe:
            total_weighted_stock[csv_name].append(tuple_stock[0])
        
        #将单独重仓股信息汇总在一张表
        total_important_stock[csv_name]=[]  ###注意必须先定义字典的值为一个列表，才能向列表里加元素!!!
        for tuple_stock in important_stock_dataframe:
            total_important_stock[csv_name].append(tuple_stock[0])
           
    total_weighted_stock = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in total_weighted_stock.items()]))
    total_important_stock = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in total_important_stock.items()]))
    
    total_weighted_stock.to_csv(total_filepath+'/'+'total_weighted_stock.csv', encoding='utf_8_sig', index=True) 
    total_important_stock.to_csv(total_filepath+'/'+'total_important_stock.csv', encoding='utf_8_sig', index=True)
    

## 并行生成网络结构数据

In [86]:
#生成网络结构数据
def xlsx_csv_fund_parallel(xlsx_filepath, hyper_params, hyper_threshold, worker = 4, invest_category = None):
    
    #创建所需文件夹
    global csv_filepath,fund_filepath,point_filepath,weighted_stock_filepath,important_stock_filepath,total_filepath
    xlsx_filepath,csv_filepath,fund_filepath,point_filepath, \
    weighted_stock_filepath,important_stock_filepath,total_filepath = mkdir(xlsx_filepath)
    
    #获取文件路径
    list_dir =  [os.path.splitext(dir_) for dir_ in os.listdir(xlsx_filepath)]
    
    #确认成的网络
    if invest_category is not None:
        print('正在生成'+'+'.join(invest_category)+'投资的网络')
    else:
        print('正在生成全部投资的网络')
    
    #生成每个线程的任务
    mission_list = sub_list(list_dir, worker)
    mul_thread = Pool(worker)
    mul_thread.map(mission_fn, mission_list)
    mul_thread.close()
    mul_thread.join() 


In [101]:
def mission_fn(list_dir):
    total_weighted_stock = {}
    total_important_stock ={}
    for tuple_dir in tqdm(list_dir):   #逐次针对当前日期
        #读取wind数据
        date_name, format_name = tuple_dir
        dataframe = pd.read_excel(xlsx_filepath+'/'+date_name+format_name)
        
        #删去不符合投资类型的股票，并检验是否数据框为空，为空则跳转到下一个循环
        if invest_category is not None:
            dataframe = dataframe[dataframe['投资类型'].isin(invest_category)]
        if dataframe.empty:
            continue
        
        #如果存在csv文件则直接读取，如果不存在则生成csv
        csv_name = ('_').join(date_name.split('-'))
        data_csv_path = csv_filepath+'/'+csv_name+'.csv'
        if not os.path.exists(data_csv_path):
            #从原始数据提取必要信息
            column_name = ['代码','名称','股票代码','股票简称','持股占流通股比(%)','持股市值占基金净值比(%)','管理公司']
            data_csv = dataframe[dataframe['代码'].apply(fliter_none)][column_name]
            data_csv = data_csv.rename(columns = {'代码':'code',
                                                  '名称':'source',
                                                  '股票代码':'stock_code',
                                                  '股票简称':'target', 
                                                  '持股占流通股比(%)':'fluent_rate',
                                                  '持股市值占基金净值比(%)':'value_rate', 
                                                  '管理公司':'group'})
            dataframe_fund = data_csv[['source','target','group','fluent_rate','value_rate']]
            
            #生成k值进一步完善dataframe_fund
            a, b, c = hyper_params
            dataframe_fund_copy = dataframe_fund.copy()
            #生成k值
            dataframe_fund_copy['k_value'] = dataframe_fund.apply(lambda row: round(a*row[3]+b*row[4]+c*row[3]*row[4], 2), axis=1)
            dataframe_fund_copy.to_csv(data_csv_path, encoding='utf_8_sig', index=True)
        else:
            print('已存在',csv_name)
            dataframe_fund_copy = pd.read_csv(data_csv_path, index_col = 0)
        
        #生成基金字典并储存基金网络
        pkl_filepath = point_filepath + '/Pickle'
        node_network_csv_path = point_filepath + '/' + csv_name + '_node_network' +'.csv'
        fund_network_pkl_path = pkl_filepath +  '/' + csv_name + '_fund_network' +'.pkl'
        if not os.path.exists(pkl_filepath):
            try:
                os.makedirs(pkl_filepath)
            except:
                continue
        if os.path.exists(node_network_csv_path):
            with open(fund_network_pkl_path,'rb') as f:   
                fund_network = pkl.load(f)   
        else:   
            fund_network = generate_dict(dataframe_fund_copy, node_network_csv_path, fund_network_pkl_path)
        fund_network_copy = deepcopy(fund_network)
        threshold = hyper_threshold
        
        #生成制作网络的数据表
        network_path = fund_filepath+'/'+csv_name+'.csv'
        if not os.path.exists(network_path):
            network_dataframe = generate_network_dataframe(fund_network,fund_network_copy,threshold)
            network_dataframe.to_csv(network_path, encoding='utf_8_sig', index=True)
        else:
            network_dataframe = pd.read_csv(network_path, index_col = 0)

        #生成共同重仓股列表
        json_weighted_filepath = weighted_stock_filepath + '/Json'
        weighted_order_csv_path = weighted_stock_filepath+'/'+csv_name+'.csv'
        weighted_order_json_path = json_weighted_filepath+'/'+csv_name+'.json'   
        fund_network_copy = deepcopy(fund_network)
        if not os.path.exists(json_weighted_filepath):
            try:
                os.makedirs(json_weighted_filepath)
            except:
                continue
        if not os.path.exists(weighted_order_csv_path):
            weighted_stock_dataframe = generate_weighted_stock(fund_network,fund_network_copy,threshold)
            with open(weighted_order_csv_path, 'w', encoding = 'utf_8_sig',newline='') as f:     
                csv_write = csv.writer(f)
                for new_line in weighted_stock_dataframe:
                    csv_write.writerow(new_line) 
            with open(weighted_order_json_path,'w', encoding = 'utf_8_sig') as f:   
                f.write(json.dumps(weighted_stock_dataframe))
        else:
            with open(weighted_order_json_path,'r', encoding = 'utf_8_sig') as f:   
                weighted_stock_dataframe = f.read()
                               ####用此方法可以将字典里的元素逐行读入；否则生成一个横向很长的列表。（转置作用）

        #生成单独重仓股列表
        json_important_filepath = important_stock_filepath + '/Json'
        important_stock_csv_path = important_stock_filepath+'/'+csv_name+'.csv'
        important_order_json_path = json_important_filepath+'/'+csv_name+'.json' 
        if not os.path.exists(json_important_filepath):
            try:
                os.makedirs(json_weighted_filepath)
            except:
                continue
        if not os.path.exists(important_stock_csv_path):
            important_stock_dataframe = generate_important_stock(fund_network,threshold)
            with open(important_stock_csv_path, 'w', encoding = 'utf_8_sig',newline='') as f:     
                csv_write = csv.writer(f)
                for new_line in important_stock_dataframe:
                    csv_write.writerow(new_line)   
            with open(important_order_json_path,'w', encoding = 'utf_8_sig') as f:   
                f.write(json.dumps(important_stock_dataframe))  
        else:
            with open(important_order_json_path,'r', encoding = 'utf_8_sig') as f:   
                important_stock_dataframe = f.read()
                
        #将共同重仓股信息汇总在一张表
        total_weighted_stock[csv_name] = ','.join([tuple_stock[0] for tuple_stock in weighted_stock_dataframe]) 
        
        #将单独重仓股信息汇总在一张表
        total_important_stock[csv_name] = ','.join([tuple_stock[0] for tuple_stock in important_stock_dataframe])  
    
    with open(total_filepath+'/'+'total_weighted_stock.csv', 'a', encoding = 'utf_8_sig') as total_weighted_stock_wirte:
        for k,v in total_weighted_stock.items():
            total_weighted_stock_wirte.writelines(','.join([k,v,'\n']))
    with open(total_filepath+'/'+'total_important_stock.csv', 'a', encoding = 'utf_8_sig') as total_important_stock_wirte:
        for k,v in total_important_stock.items():
            total_important_stock_wirte.writelines(','.join([k,v,'\n']))

## 运行主函数

In [74]:
def main_fn(xlsx_filepath,  hyper_params, hyper_threshold, invest_category, year_category, mode = 'single'):
    #生成标签文件夹来识别每一次
    #不是重新生成数据
    #是，判别年份
    #不是， 不删除数据增加数据
    #是返回数据已存在

SyntaxError: unexpected EOF while parsing (<ipython-input-74-604c1f3dd214>, line 6)

## 运行调试

In [102]:
#参数设置
xlsx_filepath = './Dataset/StockData'

hyper_params = [0.2, 0.8, 0.5]

hyper_threshold = 8 

worker = 4

#year_category = ['','']

invest_category =['普通股票型基金','指数增强型基金','灵活配置型基金','偏股混合型基金','平衡混合型基金','偏债混合型基金']

In [106]:
%%time
xlsx_csv_fund(xlsx_filepath,  hyper_params, hyper_threshold, invest_category)

已建立 ./Dataset/StockData
已建立 ./Dataset/StockData_csv
已建立 ./Dataset/StockData_fund
已建立 ./Dataset/StockData_point
已建立 ./Dataset/StockData_weighted_stock
已建立 ./Dataset/StockData_important_stock
已建立 ./Dataset/StockData_total_stock
正在生成普通股票型基金+指数增强型基金+灵活配置型基金+偏股混合型基金+平衡混合型基金+偏债混合型基金投资的网络


  0%|          | 0/73 [00:00<?, ?it/s]

Wall time: 13min 18s


In [107]:
%%time
xlsx_csv_fund_parallel(xlsx_filepath, hyper_params, hyper_threshold, worker, invest_category)

已建立 ./Dataset/StockData
已建立 ./Dataset/StockData_csv
已建立 ./Dataset/StockData_fund
已建立 ./Dataset/StockData_point
已建立 ./Dataset/StockData_weighted_stock
已建立 ./Dataset/StockData_important_stock
已建立 ./Dataset/StockData_total_stock
正在生成普通股票型基金+指数增强型基金+灵活配置型基金+偏股混合型基金+平衡混合型基金+偏债混合型基金投资的网络


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Wall time: 13min 33s
